In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Cargar los datos desde el CSV, ignorando la columna de índice
file_path = 'C:\\Users\\48242450\\Documents\\Pruebas-Proyecto\\conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis (2).csv'
df = pd.read_csv(file_path, index_col=0)

# Verificar los nombres de las columnas
print(df.columns)

# Revisar si hay valores faltantes en el dataframe
print(df.isna().sum())

# Imputar valores faltantes (puedes elegir otros métodos de imputación si lo prefieres)
imputer = SimpleImputer(strategy='mean')  # También puedes usar 'median', 'most_frequent', etc.
X = df.drop(['group', 'Initial_EDSS', 'Final_EDSS'], axis=1)
y = df['group']

# Imputar valores faltantes en X
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Hacer predicciones
y_pred = svm_model.predict(X_test)

# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Index(['Gender', 'Age', 'Schooling', 'Breastfeeding', 'Varicella',
       'Initial_Symptom', 'Mono_or_Polysymptomatic', 'Oligoclonal_Bands',
       'LLSSEP', 'ULSSEP', 'VEP', 'BAEP', 'Periventricular_MRI',
       'Cortical_MRI', 'Infratentorial_MRI', 'Spinal_Cord_MRI', 'Initial_EDSS',
       'Final_EDSS', 'group'],
      dtype='object')
Gender                       0
Age                          0
Schooling                    1
Breastfeeding                0
Varicella                    0
Initial_Symptom              1
Mono_or_Polysymptomatic      0
Oligoclonal_Bands            0
LLSSEP                       0
ULSSEP                       0
VEP                          0
BAEP                         0
Periventricular_MRI          0
Cortical_MRI                 0
Infratentorial_MRI           0
Spinal_Cord_MRI              0
Initial_EDSS               148
Final_EDSS                 148
group                        0
dtype: int64
[[20  6]
 [ 8 21]]
              precision    recall  f1-sc